In [1]:
%load_ext autoreload

In [2]:
%autoreload

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import math
import scipy as sp
import numpy as np
import pandas as pd 
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200
import seaborn as sns 
sns.set(rc={"figure.dpi":200, 'savefig.dpi':200})
sns.set_context('notebook')
sns.set_style("ticks")
import math

sns.set_style('darkgrid')

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

# from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
import sklearn as skl
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Cleaning A data

In [3]:
train_a = pd.read_parquet('data/A/train_targets.parquet').set_index("time")
len(train_a)

34085

In [4]:
train_a = pd.read_parquet('data/A/train_targets.parquet').set_index("time")
train_a = train_a[~(train_a.index < pd.Timestamp('2019-06-03 00:00:00'))]
train_a = train_a[~((train_a.index >= pd.Timestamp('2022-10-21 00:00:00')) & (train_a.index <= pd.Timestamp('2022-10-28 23:00:00')))]
train_a = train_a[~((train_a.index >= pd.Timestamp('2023-01-26 00:00:00')) & (train_a.index <= pd.Timestamp('2023-01-26 23:00:00')))]

X_train_observed_a = pd.read_parquet('data/A/X_train_observed.parquet').set_index("date_forecast")
X_train_observed_a = X_train_observed_a[~(X_train_observed_a.index < pd.Timestamp('2019-06-03 00:00:00'))]
X_train_observed_a = X_train_observed_a[~(X_train_observed_a.index >= pd.Timestamp('2022-10-21 00:00:00'))]

X_train_estimated_a = pd.read_parquet('data/A/X_train_estimated.parquet').drop("date_calc", axis=1).set_index("date_forecast")
X_train_estimated_a = X_train_estimated_a[~(X_train_estimated_a.index < pd.Timestamp('2022-10-29 00:00:00'))]

X_train_obs_est_a = pd.concat([X_train_observed_a, X_train_estimated_a])

train_datetime_interval_a = train_a.index.append(pd.DatetimeIndex([train_a.index.max() + pd.Timedelta(hours=1)]).astype("datetime64[us]"))

X_train_obs_est_a_clean_full = pd.DataFrame(columns=X_train_obs_est_a.columns)
for index in range(len(train_datetime_interval_a)-1):
    X_train_obs_est_a_clean_full_hour = X_train_obs_est_a.iloc[(X_train_obs_est_a.index >= train_datetime_interval_a[index]) & (X_train_obs_est_a.index < train_datetime_interval_a[index+1])]
    X_train_obs_est_a_clean_full = pd.concat([X_train_obs_est_a_clean_full, pd.DataFrame([X_train_obs_est_a_clean_full_hour.mean()], columns=X_train_obs_est_a_clean_full.columns)], ignore_index=True)
X_train_obs_est_a_clean_full.index = train_datetime_interval_a[:len(X_train_obs_est_a_clean_full)]
X_train_obs_est_a_clean_full.index.name = "time"


X_test_estimated_a = pd.read_parquet('data/A/X_test_estimated.parquet').drop("date_calc", axis=1).set_index("date_forecast")

test_datetime_interval_a = X_test_estimated_a.index[::4]
test_datetime_interval_a = test_datetime_interval_a.append(pd.DatetimeIndex([test_datetime_interval_a.max() + pd.Timedelta(hours=1)]).astype("datetime64[us]"))

X_test_estimated_a_clean_full = pd.DataFrame(columns=X_test_estimated_a.columns)
for index in range(len(test_datetime_interval_a)-1):
    X_test_estimated_a_clean_full_hour = X_test_estimated_a.iloc[(X_test_estimated_a.index >= test_datetime_interval_a[index]) & (X_test_estimated_a.index < test_datetime_interval_a[index+1])]
    X_test_estimated_a_clean_full = pd.concat([X_test_estimated_a_clean_full, pd.DataFrame([X_test_estimated_a_clean_full_hour.mean()], columns=X_test_estimated_a_clean_full.columns)], ignore_index=True)
X_test_estimated_a_clean_full.index = test_datetime_interval_a[:len(X_test_estimated_a_clean_full)]
X_test_estimated_a_clean_full.index.name = "time"

# Cleaning B data

In [7]:
train_b = pd.read_parquet('data/B/train_targets.parquet').set_index("time")
train_b = train_b[~(train_b.index < pd.Timestamp('2019-01-01 00:00:00'))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2019-03-31 00:00:00')) & (train_b.index < pd.Timestamp('2019-03-31 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2019-10-26 00:00:00')) & (train_b.index < pd.Timestamp('2019-10-26 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2019-10-27 00:00:00')) & (train_b.index < pd.Timestamp('2019-10-27 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2020-03-29 00:00:00')) & (train_b.index < pd.Timestamp('2020-03-29 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2020-10-24 00:00:00')) & (train_b.index < pd.Timestamp('2020-10-24 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2020-10-25 00:00:00')) & (train_b.index < pd.Timestamp('2020-10-25 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2021-03-28 00:00:00')) & (train_b.index < pd.Timestamp('2021-03-28 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2021-10-30 00:00:00')) & (train_b.index < pd.Timestamp('2021-10-30 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2021-10-31 00:00:00')) & (train_b.index < pd.Timestamp('2021-10-31 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2022-03-27 00:00:00')) & (train_b.index < pd.Timestamp('2022-03-27 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2022-05-02 00:00:00')) & (train_b.index < pd.Timestamp('2022-05-02 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2022-10-28 00:00:00')) & (train_b.index < pd.Timestamp('2022-10-28 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2022-11-03 00:00:00')) & (train_b.index < pd.Timestamp('2022-11-03 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2022-11-25 00:00:00')) & (train_b.index < pd.Timestamp('2022-11-25 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2022-11-26 00:00:00')) & (train_b.index < pd.Timestamp('2022-11-26 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2022-12-06 00:00:00')) & (train_b.index < pd.Timestamp('2022-12-06 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2022-12-27 00:00:00')) & (train_b.index < pd.Timestamp('2022-12-27 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2022-12-30 00:00:00')) & (train_b.index < pd.Timestamp('2022-12-30 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2023-01-03 00:00:00')) & (train_b.index < pd.Timestamp('2023-01-03 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2023-01-26 00:00:00')) & (train_b.index < pd.Timestamp('2023-01-26 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2023-03-07 00:00:00')) & (train_b.index < pd.Timestamp('2023-03-07 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2023-03-09 00:00:00')) & (train_b.index < pd.Timestamp('2023-03-09 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2023-03-10 00:00:00')) & (train_b.index < pd.Timestamp('2023-03-10 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2023-03-26 00:00:00')) & (train_b.index < pd.Timestamp('2023-03-26 23:00:00') + pd.Timedelta(hours=1)))]
train_b = train_b[~((train_b.index >= pd.Timestamp('2023-03-28 00:00:00')) & (train_b.index < pd.Timestamp('2023-03-28 23:00:00') + pd.Timedelta(hours=1)))]

X_train_observed_b = pd.read_parquet('data/B/X_train_observed.parquet').set_index("date_forecast")
X_train_observed_b = X_train_observed_b[~(X_train_observed_b.index >= pd.Timestamp('2022-05-03 00:00:00'))]

X_train_estimated_b = pd.read_parquet('data/B/X_train_estimated.parquet').drop("date_calc", axis=1).set_index("date_forecast")
X_train_estimated_b = X_train_estimated_b[~((X_train_estimated_b.index >= pd.Timestamp('2022-10-28 00:00:00')) & (X_train_estimated_b.index < pd.Timestamp('2022-10-28 23:00:00') + pd.Timedelta(hours=1)))]

X_train_obs_est_b = pd.concat([X_train_observed_b, X_train_estimated_b])

train_datetime_interval_b = train_b.index.append(pd.DatetimeIndex([train_b.index.max() + pd.Timedelta(hours=1)]).astype("datetime64[us]"))

X_train_obs_est_b_clean_full = pd.DataFrame(columns=X_train_obs_est_b.columns)
for index in range(len(train_datetime_interval_b)-1):
    X_train_obs_est_b_clean_full_hour = X_train_obs_est_b.iloc[(X_train_obs_est_b.index >= train_datetime_interval_b[index]) & (X_train_obs_est_b.index < train_datetime_interval_b[index+1])]
    X_train_obs_est_b_clean_full = pd.concat([X_train_obs_est_b_clean_full, pd.DataFrame([X_train_obs_est_b_clean_full_hour.mean()], columns=X_train_obs_est_b_clean_full.columns)], ignore_index=True)
X_train_obs_est_b_clean_full.index = train_datetime_interval_b[:len(X_train_obs_est_b_clean_full)]
X_train_obs_est_b_clean_full.index.name = "time"


X_test_estimated_b = pd.read_parquet('data/B/X_test_estimated.parquet').drop("date_calc", axis=1).set_index("date_forecast")

test_datetime_interval_b = X_test_estimated_b.index[::4]
test_datetime_interval_b = test_datetime_interval_b.append(pd.DatetimeIndex([test_datetime_interval_b.max() + pd.Timedelta(hours=1)]).astype("datetime64[us]"))

X_test_estimated_b_clean_full = pd.DataFrame(columns=X_test_estimated_b.columns)
for index in range(len(test_datetime_interval_b)-1):
    X_test_estimated_b_clean_full_hour = X_test_estimated_b.iloc[(X_test_estimated_b.index >= test_datetime_interval_b[index]) & (X_test_estimated_b.index < test_datetime_interval_b[index+1])]
    X_test_estimated_b_clean_full = pd.concat([X_test_estimated_b_clean_full, pd.DataFrame([X_test_estimated_b_clean_full_hour.mean()], columns=X_test_estimated_b_clean_full.columns)], ignore_index=True)
X_test_estimated_b_clean_full.index = test_datetime_interval_b[:len(X_test_estimated_b_clean_full)]
X_test_estimated_b_clean_full.index.name = "time"

# Cleaning C data

In [8]:
train_c = pd.read_parquet('data/C/train_targets.parquet').set_index("time")
train_c = train_c[~train_c.isna().any(axis=1)]
train_c = train_c[~(train_c.index < pd.Timestamp('2019-09-05 00:00:00'))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2019-10-27 00:00:00')) & (train_c.index < pd.Timestamp('2019-10-27 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2020-10-25 00:00:00')) & (train_c.index < pd.Timestamp('2020-10-25 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2021-10-31 00:00:00')) & (train_c.index < pd.Timestamp('2021-10-31 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2022-04-25 00:00:00')) & (train_c.index < pd.Timestamp('2022-04-25 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2022-05-01 00:00:00')) & (train_c.index < pd.Timestamp('2022-10-29 00:00:00')))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2022-12-05 00:00:00')) & (train_c.index < pd.Timestamp('2022-12-05 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-01-08 00:00:00')) & (train_c.index < pd.Timestamp('2023-01-08 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-01-15 00:00:00')) & (train_c.index < pd.Timestamp('2023-01-15 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-01-18 00:00:00')) & (train_c.index < pd.Timestamp('2023-01-18 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-01-26 00:00:00')) & (train_c.index < pd.Timestamp('2023-01-26 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-02-01 00:00:00')) & (train_c.index < pd.Timestamp('2023-02-01 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-02-04 00:00:00')) & (train_c.index < pd.Timestamp('2023-02-04 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-02-07 00:00:00')) & (train_c.index < pd.Timestamp('2023-02-07 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-02-09 00:00:00')) & (train_c.index < pd.Timestamp('2023-02-09 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-02-12 00:00:00')) & (train_c.index < pd.Timestamp('2023-02-12 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-02-23 00:00:00')) & (train_c.index < pd.Timestamp('2023-02-23 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-02-26 00:00:00')) & (train_c.index < pd.Timestamp('2023-02-26 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-03-05 00:00:00')) & (train_c.index < pd.Timestamp('2023-03-05 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-03-18 00:00:00')) & (train_c.index < pd.Timestamp('2023-03-18 23:00:00') + pd.Timedelta(hours=1)))]
train_c = train_c[~((train_c.index >= pd.Timestamp('2023-03-22 00:00:00')) & (train_c.index < pd.Timestamp('2023-03-22 23:00:00') + pd.Timedelta(hours=1)))]

X_train_observed_c = pd.read_parquet('data/C/X_train_observed.parquet').set_index("date_forecast")
X_train_observed_c = X_train_observed_c[~(X_train_observed_c.index < pd.Timestamp('2019-09-05 00:00:00'))]
X_train_observed_c = X_train_observed_c[~((X_train_observed_c.index >= pd.Timestamp('2019-10-27 00:00:00')) & (X_train_observed_c.index < pd.Timestamp('2019-10-27 23:00:00') + pd.Timedelta(hours=1)))]
X_train_observed_c = X_train_observed_c[~((X_train_observed_c.index >= pd.Timestamp('2020-10-25 00:00:00')) & (X_train_observed_c.index < pd.Timestamp('2020-10-25 23:00:00') + pd.Timedelta(hours=1)))]
X_train_observed_c = X_train_observed_c[~((X_train_observed_c.index >= pd.Timestamp('2021-10-31 00:00:00')) & (X_train_observed_c.index < pd.Timestamp('2021-10-31 23:00:00') + pd.Timedelta(hours=1)))]
X_train_observed_c = X_train_observed_c[~((X_train_observed_c.index >= pd.Timestamp('2022-04-25 00:00:00')) & (X_train_observed_c.index < pd.Timestamp('2022-04-25 23:00:00') + pd.Timedelta(hours=1)))]
X_train_observed_c = X_train_observed_c[~(X_train_observed_c.index >= pd.Timestamp('2022-05-01 00:00:00'))]

X_train_estimated_c = pd.read_parquet('data/C/X_train_estimated.parquet').drop("date_calc", axis=1).set_index("date_forecast")
X_train_estimated_c = X_train_estimated_c[~(X_train_estimated_c.index < pd.Timestamp('2022-10-29 00:00:00'))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2022-10-28 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2022-10-28 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2022-12-05 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2022-12-05 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-01-08 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-01-08 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-01-15 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-01-15 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-01-18 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-01-18 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-01-26 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-01-26 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-02-01 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-02-01 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-02-04 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-02-04 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-02-07 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-02-07 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-02-09 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-02-09 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-02-12 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-02-12 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-02-23 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-02-23 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-02-26 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-02-26 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-03-05 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-03-05 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-03-18 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-03-18 23:00:00') + pd.Timedelta(hours=1)))]
X_train_estimated_c = X_train_estimated_c[~((X_train_estimated_c.index >= pd.Timestamp('2023-03-22 00:00:00')) & (X_train_estimated_c.index < pd.Timestamp('2023-03-22 23:00:00') + pd.Timedelta(hours=1)))]

X_train_obs_est_c = pd.concat([X_train_observed_c, X_train_estimated_c])

train_datetime_interval_c = train_c.index.append(pd.DatetimeIndex([train_c.index.max() + pd.Timedelta(hours=1)]).astype("datetime64[us]"))

X_train_obs_est_c_clean_full = pd.DataFrame(columns=X_train_obs_est_c.columns)
for index in range(len(train_datetime_interval_c)-1):
    X_train_obs_est_c_clean_full_hour = X_train_obs_est_c.iloc[(X_train_obs_est_c.index >= train_datetime_interval_c[index]) & (X_train_obs_est_c.index < train_datetime_interval_c[index+1])]
    X_train_obs_est_c_clean_full = pd.concat([X_train_obs_est_c_clean_full, pd.DataFrame([X_train_obs_est_c_clean_full_hour.mean()], columns=X_train_obs_est_c_clean_full.columns)], ignore_index=True)
X_train_obs_est_c_clean_full.index = train_datetime_interval_c[:len(X_train_obs_est_c_clean_full)]
X_train_obs_est_c_clean_full.index.name = "time"


X_test_estimated_c = pd.read_parquet('data/C/X_test_estimated.parquet').drop("date_calc", axis=1).set_index("date_forecast")

test_datetime_interval_c = X_test_estimated_c.index[::4]
test_datetime_interval_c = test_datetime_interval_c.append(pd.DatetimeIndex([test_datetime_interval_c.max() + pd.Timedelta(hours=1)]).astype("datetime64[us]"))

X_test_estimated_c_clean_full = pd.DataFrame(columns=X_test_estimated_c.columns)
for index in range(len(test_datetime_interval_c)-1):
    X_test_estimated_c_clean_full_hour = X_test_estimated_c.iloc[(X_test_estimated_c.index >= test_datetime_interval_c[index]) & (X_test_estimated_c.index < test_datetime_interval_c[index+1])]
    X_test_estimated_c_clean_full = pd.concat([X_test_estimated_c_clean_full, pd.DataFrame([X_test_estimated_c_clean_full_hour.mean()], columns=X_test_estimated_c_clean_full.columns)], ignore_index=True)
X_test_estimated_c_clean_full.index = test_datetime_interval_c[:len(X_test_estimated_c_clean_full)]
X_test_estimated_c_clean_full.index.name = "time"

# This is where the fun begins!

In [16]:
train_a_train = train_a[train_a.index < pd.Timestamp("2023-01-25")]
train_a_test = train_a[train_a.index >= pd.Timestamp("2023-01-25")]

In [48]:
test = SARIMAX(endog = train_a_train, order = (3, 0, 1), seasonal_order=(1, 0, 1, 24))

In [49]:
test_fit = test.fit()

In [50]:
test_fit.aic

467380.1167817747

In [51]:
mean_absolute_error(test_fit.predict(start=0, end=len(train_a_test)-1), train_a_test)

764.7620007474623